# Header

text

**Note:** text

In [ ]:
import sys
###
lib_path = [r'C:\Users\ikahbasi\OneDrive\Applications\GitHub\SeisRoutine',
            r'C:\Users\ikahb\OneDrive\Applications\GitHub\SeisRoutine']
for path in lib_path:
    sys.path.append(path)

In [ ]:
import SeisRoutine.catalog as src
import SeisRoutine.waveform as srw
import SeisRoutine.config as srconf

In [ ]:
import seisbench.data as sbd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
import os

In [ ]:
import pkg_resources

# List all installed packages and their versions
installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

for package in sorted(installed_packages.keys()):
    version = installed_packages[package]
    print(f"{package}=={version}")

In [ ]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

#### Loading configuration file

In [ ]:
init_cfg = srconf.load_config('0-init-cfg.yml')
cfg = srconf.load_config(
    os.path.join(init_cfg.target_config_filepath,
                 init_cfg.target_config_filename)
)
print(cfg)

### Loading the dataset

Now that the dataset conversion is finished, we can check it by simply loading it. Here we load the dataset, print the metadata and visualize the first waveform together with the annotated pick.

In [ ]:
# base_path = Path(cfg.dataset_test.path.dataset)
# # base_path = Path(r"C:\Users\ikahb\Downloads\Kaki-Dataset-All_1403-12-04")
# data = sbd.WaveformDataset(base_path, sampling_rate=100)

In [ ]:
data = sbd.WaveformDataset(r'C:\Users\ikahbasi\Documents\Merged_All_DataSets_2025-07-10 (Ahar-Ilam-Kaki-Qeshm)', sampling_rate=100)

In [ ]:
lst = list(data.metadata.keys())

In [ ]:
list(filter(lambda x: x.startswith('trace_name'), data.metadata.keys()))

In [ ]:
data.metadata['split'] = 'train'

In [ ]:
print("Training examples:", len(data.train()))
print("Development examples:", len(data.dev()))
print("Test examples:", len(data.test()))

In [ ]:
data.metadata.keys()

In [ ]:
data.metadata['station_code'].value_counts()

In [ ]:
data.metadata['station_network_code'].value_counts()

In [ ]:
keys = data.metadata.keys()
data.metadata.head(3)[[key for key in keys if key.startswith('station')]]

In [ ]:
targets = [key for key in data.metadata.keys() if 'arrival' in key]
data.metadata[targets]
# data.metadata[['trace_Pg_snr', 'trace_Pg_snr-dB', 'trace_Sg_snr',  'trace_Sg_snr-dB', 'trace_P_snr', 'trace_gaps']]
# data.metadata['trace_Sg_snr'][1]

In [ ]:
def cmap(phase_hint):
    c = {'Pg': 'r', 'Sg': 'b', 'AML': 'c'}
    return c.get(phase_hint, 'y')

In [ ]:
range_ii = [0, 4]
for ii, metadata in data.metadata.iterrows():
    if range_ii[0] < ii <= range_ii[1]:
        # print(metadata)
        fig = plt.figure(figsize=(7, 2.5))
        ax = fig.add_subplot(111)
        trace = data.get_waveforms(ii)
        print(trace.shape)
        ax.plot(trace.T, lw=0.3)
        targets = [key for key in metadata.keys() if 'arrival' in key]
        targets = [key for key in targets if not np.isnan(metadata[key])]
        # print(targets, metadata[targets])
        for target in targets:
            phase_hint = target.split('_')[1]
            ax.axvline(metadata[target], lw=3, c=cmap(phase_hint), label=phase_hint)
        plt.legend()
        plt.show()

In [ ]:
phase_dict = {
    "trace_p_arrival_sample": "P",
    "trace_pP_arrival_sample": "P",
    "trace_P_arrival_sample": "P",
    "trace_P1_arrival_sample": "P",
    "trace_Pg_arrival_sample": "P",
    "trace_Pn_arrival_sample": "P",
    "trace_PmP_arrival_sample": "P",
    "trace_pwP_arrival_sample": "P",
    "trace_pwPm_arrival_sample": "P",
    "trace_s_arrival_sample": "S",
    "trace_S_arrival_sample": "S",
    "trace_S1_arrival_sample": "S",
    "trace_Sg_arrival_sample": "S",
    "trace_SmS_arrival_sample": "S",
    "trace_Sn_arrival_sample": "S",
}

In [ ]:
tmp = {}
for key, val in phase_dict.items():
    tmp[key.replace('trace', 'trace_manual')] = val

phase_dict = tmp

In [ ]:
phase_dict

In [ ]:
import seisbench.generate as sbg

In [ ]:
generator = sbg.GenericGenerator(data)

augmentations = [
    sbg.WindowAroundSample(list(phase_dict.keys()), samples_before=3000, windowlen=6000, selection="random", strategy="variable"),
    sbg.RandomWindow(windowlen=3001, strategy="pad"),
    sbg.Normalize(demean_axis=-1, amp_norm_axis=-1, amp_norm_type="peak"),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(label_columns=phase_dict, sigma=30, dim=0)
]

generator.add_augmentations(augmentations)

In [ ]:
import numpy as np
from scipy import signal

class Tapering:
    def __init__(self, alpha=0.3, key='X'):
        self.alpha = alpha  # ضریب تیپرینگ
        if isinstance(key, str):
            self.key = (key, key)
        else:
            self.key = key

    def __call__(self, state_dict):
        x, metadata = state_dict[self.key[0]]
        taper = signal.windows.tukey(x.shape[-1], self.alpha)
        x = x * taper
        state_dict[self.key[1]] = (x, metadata)


In [ ]:
generator = sbg.GenericGenerator(data)

augmentations = [
    Tapering(),
    # sbg.WindowAroundSample(metadata_keys=list(phase_dict.keys()), samples_before=6000, windowlen=2*6000, selection="random", strategy="variable"),
    sbg.RandomWindow(windowlen=2*3001, strategy="pad"),
    sbg.GaussianNoise(scale=(0.01, 0.02), key='X'),
    # sbg.RealNoise(noise_dataset, scale=(0, 1), scaling_type='peak', metadata_thresholds=None, key='X'),
    sbg.RandomWindow(windowlen=3001, strategy="pad"),
    sbg.Normalize(demean_axis=-1, amp_norm_axis=-1, amp_norm_type="peak"),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(label_columns=phase_dict, sigma=30, dim=0)
]

generator.add_augmentations(augmentations)

In [ ]:
def plot_generator(n=None):
    if n is None:
        n = np.random.randint(len(generator))
    print(f'{n=}')
    print(data.metadata.iloc[n])
    sample = generator[n]
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(15, 5),
                            gridspec_kw={"hspace": 0, "height_ratios": [3, 1]})
    axs[0].plot(sample["X"].T)
    axs[1].plot(sample["y"].T)

In [ ]:
plot_generator(n=3347)

In [ ]:
plot_generator(n=0)

In [ ]:
plot_generator(n=15229)

In [ ]:
plot_generator(n=15351)

## Considerations for converting datasets

As outlined above, this tutorial provides a very minimal example on converting a dataset. Here we outline additional consideration that should be taken into account when preparing a dataset.

- **Grouping picks**: In this example, we created one trace for each pick. Naturally, traces will overlap if multiple picks, e.g., P and S phases, are available for an event at a station. For an example implementation of this grouping operation, have a look [here](https://github.com/seisbench/seisbench/blob/df94dcd86ce66d6a2ee2bd00da3857259fe579bd/seisbench/data/ethz.py#L109) and in the subsequent lines.
- **Adding station information**: In this example, we added no station information except its name. In practice, it will often be helpful for users to incorporate, for example, the location of the station. We skipped this step here, because it requires loading station inventories through FDSN. For an example implementation, have a look [here](https://github.com/seisbench/seisbench/blob/df94dcd86ce66d6a2ee2bd00da3857259fe579bd/seisbench/data/ethz.py#L315).
- **Memory requirements**: Internally, the `WaveformDataWriter` writes out the the waveforms continuously in blocks (see point below), but keeps all metadata in memory until the dataset is complete. For very large datasets (or very detailed metadata) this can result in several gigabytes of memory consumption. If you are writing such datasets, make sure the available memory on your machine is sufficient.
- **Waveform blocks**: Instead of writing each waveform separately, waveforms are written out in blocks. This massively improves IO performance. Have a look at [the documentation](https://seisbench.readthedocs.io/en/stable/pages/data_format.html#traces-blocks) for details on the strategy. We expect that in nearly all cases using the default setting will be a good choice.
- **FDSN considerations**: When converting very large datasets, the performance might be limited by the performance of the FDSN webservice. Unfortunately, downloading lots of short waveforms (as required for many machine learning applications) does not seem to be the most favorable use case for FDSN. This leads to rather slow performance when naively downloading the waveforms as outlined above. Instead, it is often helpful to issue [bulk requests](https://docs.obspy.org/master/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms_bulk.html). In addition, it might be a good choice to first download the waveforms and cache them locally, for example, in .mseed format, and then convert them to SeisBench.

For further details on the data format, check out [the data format specification in the SeisBench documentation](https://seisbench.readthedocs.io/en/stable/pages/data_format.html#traces-blocks).

In [ ]:
ii = 4
metadata = data.metadata.iloc[ii]
# print(metadata)
fig, axes = plt.subplots(3, 1, figsize=(10, 5), sharex=True)
trace = data.get_waveforms(ii)
print(trace.shape)
for jj in range(3):
    axes[jj].plot(trace.T[:, jj], c='k', lw=0.5)
    axes[jj].patch.set_visible(False)
    axes[jj].axis('off')
targets = [key for key in metadata.keys() if 'arrival' in key]
targets = [key for key in targets if not np.isnan(metadata[key])]
# print(targets, metadata[targets])
for target in targets:
    phase_hint = target.split('_')[1]
    for jj in range(3):
        axes[jj].axvline(metadata[target], lw=2, c=cmap(phase_hint), label=phase_hint[0])
axes[1].legend()

# Test

In [ ]:
class Tapering:
    def __init__(self, alpha=0.2, key='X'):
        self.alpha = alpha  # ضریب تیپرینگ
        if isinstance(key, str):
            self.key = (key, key)
        else:
            self.key = key

    def __call__(self, state_dict):
        x, metadata = state_dict[self.key[0]]
        taper = signal.windows.tukey(x.shape[-1], self.alpha)
        x = x * taper
        state_dict[self.key[1]] = (x, metadata)

In [ ]:
augmentations = [
    sbg.ProbabilisticLabeller(label_columns=phase_dict,
                              model_labels=cfg.model.phases,
                              sigma=30, dim=0),    
]

generator = sbg.GenericGenerator(data.train())
generator.add_augmentations(augmentations)


sample_number = 1124 #np.random.randint(len(generator))
sample = generator[sample_number]

fig = plt.figure(figsize=(15, 5))
axs = fig.subplots(2, 1, sharex=True, gridspec_kw={"hspace": 0, "height_ratios": [3, 1]})
axs[0].plot(sample["X"].T)
axs[1].plot(sample["y"].T)
plt.suptitle(sample_number)

In [ ]:
augmentations = [
    Tapering(),
    sbg.FixedWindow(p0=-35*100, windowlen=2*3000, strategy="pad", key='X'),
    sbg.Normalize(demean_axis=-1, amp_norm_axis=-1, amp_norm_type="peak"),
    # sbg.GaussianNoise(scale=(0, 0.002), key='X'),
    sbg.RandomWindow(windowlen=3001, strategy="pad"),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(label_columns=phase_dict,
                              model_labels=cfg.model.phases,
                              sigma=30, dim=0),    
]

train = data.train()
generator = sbg.GenericGenerator(train)
generator.add_augmentations(augmentations)


sample_number = 432#np.random.randint(len(generator))
sample = generator[sample_number]

fig = plt.figure(figsize=(15, 5))
axs = fig.subplots(2, 1, sharex=True, gridspec_kw={"hspace": 0, "height_ratios": [3, 1]})
axs[0].plot(sample["X"].T)
axs[1].plot(sample["y"].T)
plt.suptitle(sample_number)

In [ ]:
augmentations = [
    Tapering(),
    sbg.FixedWindow(p0=-35*100, windowlen=2*3000, strategy="pad", key='X'),
    sbg.Normalize(demean_axis=-1, amp_norm_axis=-1, amp_norm_type="peak"),
    # sbg.GaussianNoise(scale=(0, 0.002), key='X'),
    sbg.RandomWindow(windowlen=3001, strategy="pad"),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(label_columns=phase_dict,
                              model_labels=cfg.model.phases,
                              sigma=30, dim=0),    
]

train = data.train()
generator = sbg.GenericGenerator(train)
generator.add_augmentations(augmentations)


sample_number = 432#np.random.randint(len(generator))
sample = generator[sample_number]

fig = plt.figure(figsize=(15, 5))
axs = fig.subplots(2, 1, sharex=True, gridspec_kw={"hspace": 0, "height_ratios": [3, 1]})
axs[0].plot(sample["X"].T)
axs[1].plot(sample["y"].T)
plt.suptitle(sample_number)